In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
file_name = 'timestamp2Clatitude2Cbattery_vo.csv'
df = pd.read_csv(file_name)
print("Initial Data:")
print(df.head())
df.replace('', np.nan, inplace=True)
print(f"\nOriginal dataset shape: {df.shape}")

Initial Data:
   timestamp  altitude_m  latitude  longitude  battery_voltage flight_mode
0          0         0.2   13.3490    74.7421             12.6      MANUAL
1          1         5.1   13.3492    74.7423             12.5        AUTO
2          2        10.3   13.3494    74.7425             12.4        AUTO
3          3        14.8   13.3496    74.7427             12.2        AUTO
4          4        20.1   13.3498    74.7429             12.1         NaN

Original dataset shape: (21, 6)


In [53]:
print("\nRemoving Noise part")

# Create new DataFrame keeping only rows with altitude less than 1000
df_cleaned = df[df['altitude_m'] <= 1000].copy()
# From the new DataFrame, keep only rows with battery_voltage > 1
df_cleaned = df_cleaned[df_cleaned['battery_voltage'] > 1]
df_cleaned = df_cleaned[df_cleaned['altitude_m'] > 0]
rows_removed = df.shape[0] - df_cleaned.shape[0]
print(f"Removed {rows_removed} rows based on noise conditions.")


Removing Noise part
Removed 4 rows based on noise conditions.


In [54]:
print("\nMissing values *before* filling:")
print(df_cleaned.isnull().sum())

df_cleaned['altitude_m'] = df_cleaned['altitude_m'].ffill()
df_cleaned['flight_mode'] = df_cleaned['flight_mode'].ffill()
df_cleaned['latitude'] = df_cleaned['latitude'].ffill()      
df_cleaned['longitude'] = df_cleaned['longitude'].ffill()     

print("\nMissing values *after* forward-fill:")
print(df_cleaned.isnull().sum())

df_cleaned.dropna(inplace=True)

print(f"Final dataset shape after dropping NaNs: {df_cleaned.shape}")


Missing values *before* filling:
timestamp          0
altitude_m         0
latitude           1
longitude          1
battery_voltage    0
flight_mode        1
dtype: int64

Missing values *after* forward-fill:
timestamp          0
altitude_m         0
latitude           0
longitude          0
battery_voltage    0
flight_mode        0
dtype: int64
Final dataset shape after dropping NaNs: (17, 6)


In [55]:
print("Generating Altitude vs. Timestamp plot")
plt.figure(figsize=(10, 6))
plt.plot(df_cleaned['timestamp'], df_cleaned['altitude_m'])
plt.xlabel('Timestamp')
plt.ylabel('Altitude (m)')
plt.title('Drone Altitude vs. Timestamp (Cleaned)')
plt.grid(True) # Add a grid

plt.savefig('altitude_vs_timestamp.png')
plt.clf()
print("Saved 'altitude_vs_timestamp.png'")

Generating Altitude vs. Timestamp plot
Saved 'altitude_vs_timestamp.png'


<Figure size 1000x600 with 0 Axes>

In [57]:
print("Flight Path plot")
plt.figure(figsize=(10, 8))

modes = df_cleaned['flight_mode'].unique()
colors = plt.cm.get_cmap('tab10', len(modes))
mode_color_map = {mode: colors(i) for i, mode in enumerate(modes)}

for mode in modes:
    mode_data = df_cleaned[df_cleaned['flight_mode'] == mode]
    plt.scatter(
        mode_data['longitude'],       
        mode_data['latitude'],        
        label=mode,                   
        color=mode_color_map[mode],   
        s=50,                         
        alpha=0.8                     
    )
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Drone Flight Path by Flight Mode')

plt.legend(title='Flight Mode')
plt.grid(True)
plt.axis('equal')

plt.savefig('flight_path.png')
plt.clf()
print("Saved 'flight_path.png'")

Flight Path plot


C:\Users\Atharv\AppData\Local\Temp\ipykernel_7656\1735139959.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(modes))


Saved 'flight_path.png'


<Figure size 1000x800 with 0 Axes>